#**TO DO**
- train the god damn model
- try old school generator

# **Download & Unzipping COCO dataset (USE ONLY FROM CPU TAB)**

In [ ]:
#Download Training Data
!wget http://images.cocodataset.org/zips/train2017.zip
!unzip /content/train2017.zip
!rm /content/train2017.zip

#Download Validation Data
!wget http://images.cocodataset.org/zips/val2017.zip
!unzip /content/val2017.zip
!rm /content/val2017.zip

#Download Testing Data
!wget http://images.cocodataset.org/zips/test2017.zip
!unzip /content/test2017.zip
!rm /content/test2017.zip

#Download annotation for Training and Validation Data
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip /content/annotations_trainval2017.zip
!rm /content/annotations_trainval2017.zip

# delete the sameple folder data 
%rm -rf /content/sample_data

# ***Import Libraries***

In [ ]:
import os
import cv2
import json
import string
import numpy as np
import pickle as pkl
from random import shuffle
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow as imshow

import keras
import tensorflow as tf
from keras import Model
from keras.layers.merge import add
from keras.models import Sequential 
from keras.utils.vis_utils import plot_model
from keras.utils import np_utils, data_utils
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.image import ImageDataGenerator , load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.merge import Concatenate
from keras.layers import Dense, Conv2D, LSTM, Embedding ,Dropout, Activation, Input 

## **Downloading Data (USE ONLY FROM GPU TAB)**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Download annotation for Training and Validation Data
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip /content/annotations_trainval2017.zip
!rm /content/annotations_trainval2017.zip

# copy resized images from drive
!cp -r "/content/drive/MyDrive/Colab Notebooks/Project 2/Original/Content Folder/" "/content/"

!unzip "/content/Content Folder/train2017.zip" -d "/"
!unzip "/content/Content Folder/val2017.zip" -d "/"
!unzip "/content/Content Folder/test2017.zip" -d "/"

%rm -rf "/content/Content Folder"

# **Creating Custom Tokenizer**
- **remove rare words**
- **replace rare word with "\<UNK>" token**

In [ ]:
class Tokenizer():
  def __init__(self):
    self.VOCB_SIZE = 0
    
    self.deleted_words = list()
   
    self.word2int = dict()
    self.word2int['<UNK>'] = 0
    
    self.int2word = dict()
 
  def __len__(self):
    return self.VOCB_SIZE

  def fit_on_texts(self,all_text):
    words_to_delete = list()
    count_dictionary = dict()
    
    #count how many each word appears
    for line in all_text:
      for word in line:
        if word not in count_dictionary:
          count_dictionary[word] = 0  
        
        count_dictionary[word]+=1
    
    #prepare words to be deleted of it appears less than 2 times
    for key,value in count_dictionary.items():
      if value <2:
        words_to_delete.append(key)

    #delete rare words
    for word in words_to_delete:
      del count_dictionary[word]
    
    #create word to number dictionary
    i = 1
    for line in all_text:
      for word in line:
        if word in count_dictionary and word not in self.word2int:
          self.word2int[word] = i
          i+=1

    
    #create number to word dictionary
    self.int2word = {num:word for word,num in self.word2int.items()}

    self.VOCB_SIZE = len(list(self.word2int.items()))
    self.deleted_words = words_to_delete

  def texts_to_sequences(self,text):
    seq = list()
    for word in text:
      if word in self.deleted_words:
        seq.append(0)
      else:
        seq.append(self.word2int[word])
    
    return seq
  
  def sequences_to_text(self,sequence):
    txt = [self.int2word(num) for num in sequence]
    return txt




# **Prepare Data Paths**

In [ ]:
origin_train_dir = '/content/train2017/'
origin_test_dir = '/content/test2017/'
origin_val_dir = '/content/val2017/'
caption_train_file_dir = '/content/annotations/captions_train2017.json'
caption_val_file_dir = '/content/annotations/captions_val2017.json'

#**Resizing to 224 for ResNet50 (USE ONLY FROM CPU TAB)**

 - **Resnet50 accept input of (224,224,3)**
 - **To free space I overwrite all images with its new size (224,224)**
 - **This will also speed up loading images instead of resizing every epoch**

In [ ]:
def resize_to_224(img_dir):
  img = cv2.imread(img_dir)
  img = cv2.resize(img,(224,224))
  cv2.imwrite(img_dir,img)


In [ ]:
for path in os.listdir(origin_train_dir):
  resize_to_224(origin_train_dir+path)


for path in os.listdir(origin_val_dir):
  resize_to_224(origin_val_dir+path)

for path in os.listdir(origin_test_dir):
  resize_to_224(origin_test_dir+path)

**zipping files to be passed to gpu**

In [ ]:
!zip -r /content/val2017.zip /content/val2017
!zip -r /content/train2017.zip /content/train2017
!zip -r /content/test2017.zip /content/test2017

#**Cleaning Caption**
- **Ronvert all words to lower case to avoid same seeing the word differently**
- **Remove any punctuation like (<, %, #, ... etc)**
- **Remove any word with 1 letter ( WE ARE LOOKING FOR WORDS !!!! بدهيات)**
- **Remove any numeric query keep just alphabetic queries**

In [ ]:
def clean_text(caption):
  table = str.maketrans(" "," ",string.punctuation)
  caption = caption.split()
  
  caption = [word.lower() for word in caption]
  caption = [word.translate(table) for word in caption]
  caption = [word for word in caption if len(word)>1]  
  caption = [word for word in caption if word.isalpha()]
  

  return caption

#**Loading images with corresponding caption in a Dictinary**
- **Keys are image paths**
- **Values are a list of corresponding captions for each Image**
- **it also return a list of all captions compined for future use**

In [ ]:
def extract_data (caption_dir):
  
  img_to_caption = dict()
  all_text = list()

  with open(caption_dir, 'r') as f:
    jsonfile = json.load(f)
    annotations = jsonfile['annotations']

  for annotation in annotations:
    img_id = annotation['image_id']
    caption = clean_text(annotation['caption'])
    
    #Add "start" at first of the caption
    caption.insert(0,"<start> ")
    
    #and adding "end" at the end of the caption
    caption.append(" <end>")
    
    #remove any caption longer than 35 words + start and end words
    if len(caption) > 37:
      continue

    all_text.append(caption)

    img_id =  "%012d" % (img_id)   

    if img_id not in img_to_caption:
      img_to_caption[img_id] = list()
    img_to_caption[img_id].append(caption)    

  img_to_caption_list = [ [k,v] for k,v in img_to_caption.items()]
  return img_to_caption_list , all_text

#**Extracting Data**

In [ ]:
train_list ,  train_text = extract_data(caption_train_file_dir)
val_list , val_text = extract_data(caption_val_file_dir)

all_text = train_text + val_text

size_of_training_data = len(train_list)
size_of_val_data = len(val_list)

print("Number of training elements %d" %size_of_training_data)
print("Number of validation elements %d" %size_of_val_data)

Number of training elements 118287
Number of validation elements 5000


#**Unique words to tokens**
- **make a tokenizer of words where each word only appear once**
- **each word will have a number which is non repetative**
- **it will be used to generate numbers (model understand numbers only)  from every caption**

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_text)
 
VOCAB_SIZE = len(tokenizer)


print("Number of unique words found %d" %VOCAB_SIZE)

Number of unique words found 16759


#**Find Longest Caption**

In [ ]:
def max_line_lenght(all_text):
  return max(len(line) for line in all_text )


MAX_CAPTION_LENGHT = max_line_lenght(all_text)


print("Size of longest caption line %d" %MAX_CAPTION_LENGHT)

Size of longest caption line 37


#**Getting the Feature map Images with ResNet50**
- **Passing that image to ResNet and getting output Feature Map with** `extract_feature_map`
- **Saving feature Map to disk with** `save_feature_map`
- **getting Feature Map from disk with** `get_feature_map`

In [ ]:
EncoderNet = ResNet50(input_tensor=Input(shape=(224 , 224 , 3)),weights='imagenet',include_top=False,pooling='avg')
EncoderNet.trainable = False

def extract_feature_map(image_path):
  #assert size is (224,224) avoid any error
  img = load_img(image_path,target_size=(224,224))

  # add one dim at first (1,244,244,3) it will be used for batch size in future
  img = np.expand_dims(img, axis=0)
  img = EncoderNet.predict(img)
  img = np.reshape(img, img.shape[1])
  return img


94781440/94765736 [==============================] - 1s 0us/step


**Better use with GPU tab for faster process**

In [ ]:
def loading_to_memory(data_list,folder):
  for i , item in enumerate(data_list):
    dst_dir = folder + item[0] + '.jpg'
    img = extract_feature_map(dst_dir)

    data_list[i][0] = img
  
  return data_list

train_list = loading_to_memory(train_list,origin_train_dir)
val_list = loading_to_memory(val_list,origin_val_dir)

1000 f_maps have passed
2000 f_maps have passed
3000 f_maps have passed
4000 f_maps have passed
5000 f_maps have passed
6000 f_maps have passed
7000 f_maps have passed
8000 f_maps have passed
9000 f_maps have passed
10000 f_maps have passed
11000 f_maps have passed
12000 f_maps have passed
13000 f_maps have passed
14000 f_maps have passed
15000 f_maps have passed
16000 f_maps have passed
17000 f_maps have passed
18000 f_maps have passed
19000 f_maps have passed
20000 f_maps have passed
21000 f_maps have passed
22000 f_maps have passed
23000 f_maps have passed
24000 f_maps have passed
25000 f_maps have passed
26000 f_maps have passed
27000 f_maps have passed
28000 f_maps have passed
29000 f_maps have passed
30000 f_maps have passed
31000 f_maps have passed
32000 f_maps have passed
33000 f_maps have passed
34000 f_maps have passed
35000 f_maps have passed
36000 f_maps have passed
37000 f_maps have passed
38000 f_maps have passed
39000 f_maps have passed
40000 f_maps have passed
41000 f_m

# **Saving/Loading all important files to Magic file**
**This file contains:**
- training_list, val_list
- all_text_caption
- tokenizer
- VOCAB_SIZE
- MAX_CAPTION_LENGHT
- size_of_training_data
- size_of_val_data


**SAVING**

In [ ]:
with open("/content/MagicFile.F", "wb") as f :
  pkl.dump(train_list,f)
  pkl.dump(val_list,f)
  pkl.dump(all_text,f)
  pkl.dump(tokenizer,f)
  pkl.dump(VOCAB_SIZE,f)
  pkl.dump(MAX_CAPTION_LENGHT,f)
  pkl.dump(size_of_training_data,f)
  pkl.dump(size_of_val_data,f)